# Azure functions example

This notebook shows how to use the function calling capability with the Azure OpenAI service. Functions allow a caller of chat completions to define capabilities that the model can use to extend its
functionality into external tools and data sources.

You can read more about chat functions on OpenAI's blog: https://openai.com/blog/function-calling-and-other-api-updates

**NOTE**: Chat functions require model versions beginning with gpt-4 and gpt-35-turbo's `-0613` labels. They are not supported by older versions of the models.

## Setup

First, we install the necessary dependencies and import the libraries we will be using.

In [136]:
! pip install "openai>=1.0.0,<2.0.0"
! pip install python-dotenv


[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [137]:
import os
import openai
import dotenv

dotenv.load_dotenv()

True

### Authentication

The Azure OpenAI service supports multiple authentication mechanisms that include API keys and Azure Active Directory token credentials.

In [138]:
use_azure_active_directory = False  # Set this flag to True if you are using Azure Active Directory

#### Authentication using API key

To set up the OpenAI SDK to use an *Azure API Key*, we need to set `api_key` to a key associated with your endpoint (you can find this key in *"Keys and Endpoints"* under *"Resource Management"* in the [Azure Portal](https://portal.azure.com)). You'll also find the endpoint for your resource here.

In [139]:
if not use_azure_active_directory:
    endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
    api_key = os.environ["AZURE_OPENAI_API_KEY"]

    client = openai.AzureOpenAI(
        azure_endpoint=endpoint,
        api_key=api_key,
        api_version="2023-09-01-preview"
    )

#### Authentication using Azure Active Directory
Let's now see how we can autheticate via Azure Active Directory. We'll start by installing the `azure-identity` library. This library will provide the token credentials we need to authenticate and help us build a token credential provider through the `get_bearer_token_provider` helper function. It's recommended to use `get_bearer_token_provider` over providing a static token to `AzureOpenAI` because this API will automatically cache and refresh tokens for you. 

For more information on how to set up Azure Active Directory authentication with Azure OpenAI, see the [documentation](https://learn.microsoft.com/azure/ai-services/openai/how-to/managed-identity).

In [140]:
! pip install "azure-identity>=1.15.0"


[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [141]:
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

if use_azure_active_directory:
    endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
    api_key = os.environ["AZURE_OPENAI_API_KEY"]

    client = openai.AzureOpenAI(
        azure_endpoint=endpoint,
        azure_ad_token_provider=get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default"),
        api_version="2023-09-01-preview"
    )

> Note: the AzureOpenAI infers the following arguments from their corresponding environment variables if they are not provided:

- `api_key` from `AZURE_OPENAI_API_KEY`
- `azure_ad_token` from `AZURE_OPENAI_AD_TOKEN`
- `api_version` from `OPENAI_API_VERSION`
- `azure_endpoint` from `AZURE_OPENAI_ENDPOINT`


## Deployments

In this section we are going to create a deployment of a GPT model that we can use to call functions.

### Deployments: Create in the Azure OpenAI Studio
Let's deploy a model to use with chat completions. Go to https://portal.azure.com, find your Azure OpenAI resource, and then navigate to the Azure OpenAI Studio. Click on the "Deployments" tab and then create a deployment for the model you want to use for chat completions. The deployment name that you give the model will be used in the code below.

In [142]:
deployment = "gpt-4" # Fill in the deployment name from the portal here

## Functions

With setup and authentication complete, you can now use functions with the Azure OpenAI service. This will be split into a few steps:

1. Define the function(s)
2. Pass function definition(s) into chat completions API
3. Call function with arguments from the response
4. Feed function response back into chat completions API

#### 1. Define the function(s)

A list of functions can be defined, each containing the name of the function, an optional description, and the parameters the function accepts (described as a JSON schema).

In [143]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_geo_location",
            "description": "Get the geo location of a city as string 'latitude,longitude'. Parameters state and country are optional. Result e.g. '25.245470718844146,51.45400942457904'",
            "parameters": {
                "type": "object",
                "properties": {
                    "country": {
                        "type": "string",
                        "description": "The country code in format 'ISO 3166-1 alpha-2', e.g. US",
                    },
                    "state": {
                        "type": "string",
                        "description": "The state, e.g. CA",
                    },                    
                    "city": {
                        "type": "string",
                        "description": "The city, e.g. San Francisco",
                    }
                },
                "required": ["city"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather at a geo location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The geo location as string 'latitude,longitude', e.g. '25.245470718844146,51.45400942457904'",
                    },
                    "format": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the users location.",
                    },
                },
                "required": ["location", "format"],
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_n_day_weather_forecast",
            "description": "Get an N-day weather forecast for a geo location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The geo location as string 'latitude,longitude', e.g. '25.245470718844146,51.45400942457904'",
                    },
                    "format": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the users location.",
                    },
                    "num_days": {
                        "type": "integer",
                        "description": "The number of days to forecast",
                    }
                },
                "required": ["location", "format", "num_days"]
            },
        }
    },
]

#### 2. Declare functions for prompt chat

In [144]:
def get_geo_location(request):
    """
    Get country, state, and city from the user's request.
    Create response object with dummy location property for illustrative purposes.
    """
    country = request.get("country")
    state = request.get("state")
    city = request.get("city")
    return {"location": "25.245470718844146,51.45400942457904"}

def get_current_weather(request):
    """
    This function is for illustrative purposes.
    The location and unit should be used to determine weather
    instead of returning a hardcoded response.
    """
    location = request.get("location")
    format = request.get("format")
    return {"temperature": "22", "format": "celsius", "description": "Sunny"}

def get_n_day_weather_forecast(request):
    """
    This function is for illustrative purposes.
    The location and unit should be used to determine weather
    instead of returning a hardcoded response.
    """
    location = request.get("location")
    format = request.get("format")
    num_days = request.get("num_days")
    return {"temperature": "30", "format": "farenheit", "description": "Sunny", "num_days": num_days}

#### 3. Pass function definition(s) into chat completions API

Now we can pass the function into the chat completions API. If the model determines it should call the function, a `finish_reason` of "tool_calls" will be populated on the choice and the details of which function to call and its arguments will be present in the `message`. Optionally, you can set the `tool_choice` keyword argument to force the model to call a particular function (e.g. `{"type": "function", "function": {"name": get_current_weather}}`). By default, this is set to `auto`, allowing the model to choose whether to call the function or not. 

In [145]:
import json

messages = [
    {"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."},

# Exact function declaration is important to detect the functions and optional parameters, not ending in infinitive loop calling the same function!

# User messages to test the bot. Country and state are optional.

# Chaining single ok, first get_geo_location, then get_current_weather
    {"role": "user", "content": "What's the weather like today in Seattle, WA, US?"}

# Chaining single ok, first get_geo_location, then get_n_day_weather_forecast
#    {"role": "user", "content": "What's the weather like in 20 days in Seattle, WA, US?"}

# Chaining single ok, first get_geo_location, then get_current_weather
#    {"role": "user", "content": "What's the weather like today in Seattle?"}

# Chaining multiple ok first get_geo_location, then get_current_weather
#    {"role": "user", "content": "What's the weather like today in the following cities: Seattle and New York?"}

# Chaining single ok, first get_geo_location, then get_n_day_weather_forecast
#    {"role": "user", "content": "What's the weather like in 20 days in Seattle?"}

# Chaining multiple ok, first get_geo_location, then get_n_day_weather_forecast
#    {"role": "user", "content": "What's the weather like? In 20 days in Seattle and in 30 days in New York?"}

# Chaining multiple mixed ok, first get_geo_location, then get_current_weather / get_n_day_weather_forecast
#    {"role": "user", "content": "What's the weather like? Today in Seattle and in 30 days in New York?"}

# Direct call single ok: get_current_weather
#    {"role": "user", "content": "What's the weather like today at 25.245470718844146,51.45400942457904?"},

# Direct call multiple ok: get_current_weather
#    {"role": "user", "content": "What's the weather like today at 25.245470718844146,51.45400942457904 and at 52.245470718844146,15.45400942457904?"},

# Direct call single ok: get_n_day_weather_forecast
#    {"role": "user", "content": "What's the weather like in 15 days at 25.245470718844146,51.45400942457904?"}

# Direct call multiple ok: get_n_day_weather_forecast
#    {"role": "user", "content": "What's the weather like? In 15 days at 25.245470718844146,51.45400942457904? In 30 days at 52.245470718844146,15.45400942457904?"}

# Direct call multiple mixed ok: get_geo_location, get_current_weather, get_n_day_weather_forecast
#    {"role": "user", "content": "What's the weather like? Today in New York? Tomorrow in London? In 15 days at 25.245470718844146,51.45400942457904? In 30 days at 52.245470718844146,15.45400942457904?"}

# result: single confusion: Seattle is in WA, US
#    {"role": "user", "content": "What's the weather like in 20 days in Seattle, WA, CA?"}

# result: mixed confusion: Seattle is in WA, US - but London is ok - still all confused and no function call
#    {"role": "user", "content": "What's the weather like in 20 days in London and Seattle, WA, CA?"}

# result: mixed confusion: Seattle is in WA, US - but London is ok - still all confused and no function call
#    {"role": "user", "content": "What's the weather like? In 20 days in London? Today in Seattle, WA, CA?"}
]

print ("- Initial messages: ")
print (json.dumps(messages))

chat_completion = client.chat.completions.create(
    model=deployment,
    messages=messages,
    tools=tools,
)
print ("- Initial response: ")
print(chat_completion)

- Initial messages: 
[{"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."}, {"role": "user", "content": "What's the weather like? In 20 days in London? Today in Seattle, WA, CA?"}]


- Initial response: 
ChatCompletion(id='chatcmpl-8n26odTb14SpbbBnzMVSIT4YX2Acz', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='To clarify, do you want to know the weather for Seattle in the state of Washington or the city in California?', role='assistant', function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}})], created=1706696870, model='gpt-4', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=24, prompt_tokens=356, total_tokens=380), prompt_filter_results=[{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered':

#### 4. Call function with arguments from the response and feed function response back into chat completions API

The name of the function call will be one that was provided initially and the arguments will include JSON matching the schema included in the function definition.

The response from the function should be serialized into a new message with the role set to "function". Now the model will use the response data to formulate its answer.

In [146]:
import json
auto_interaction_max = 10
auto_interaction_counter = 0

# loop until either chat_completion has no choices or the first choice is no function
while auto_interaction_counter < auto_interaction_max and len(chat_completion.choices) > 0 and chat_completion.choices[0].message.tool_calls:
    auto_interaction_counter += 1

    print ("\n### Processing {}. automatic interaction ##########################################".format(auto_interaction_counter))

    # extend conversation with assistant's reply to avoid error code 400: Invalid parameter: messages with role 'tool' must be a response to a preceeding message with 'tool_calls'.
    messages.append(chat_completion.choices[0].message)  

    # Process all function calls 
    for tool_call in chat_completion.choices[0].message.tool_calls:
        function_name = tool_call.function.name
        function_arguments = tool_call.function.arguments
        print("\n### Call function: {} [ID: {}]: {}".format(function_name, tool_call.id, function_arguments.replace('\n', ' ').replace('\r', '')))

        # Process function call
        unknown_tool = False
        if function_name == "get_current_weather":
            tool_result = get_current_weather(json.loads(function_arguments))
        elif function_name == "get_n_day_weather_forecast":
            tool_result = get_n_day_weather_forecast(json.loads(function_arguments))
        elif function_name == "get_geo_location":
            tool_result = get_geo_location(json.loads(function_arguments))
        else:
            tool_result = ""
            unknown_tool = True

        # Print function result            
        if unknown_tool:
            print("- Unknown Function: {}".format(function_name))
        else:
            print("- Function Result: {}".format(tool_result))

            # Append tool call id to result
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": json.dumps(tool_result)
                }
            )

    # List overall updated message and do reprocessing
    print ("- Updated messages: {}".format(messages))

    chat_completion = client.chat.completions.create(
        model=deployment,
        messages=messages,
        tools=tools,
    )

# Final chat result without function calls
print ("\n### Final message ##########################################")
if chat_completion.choices[0].message.content:
    print(chat_completion.choices[0].message.content.strip())
else:
    print(chat_completion.choices)


### Final message ##########################################
To clarify, do you want to know the weather for Seattle in the state of Washington or the city in California?
